# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns # Removed due to import error
from scipy import stats

df = pd.read_csv('homepage_actions.csv')
display(df.head())

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
print("Dataset Info:")
print(df.info())
print("\nGroup Counts:")
print(df['group'].value_counts())
print("\nAction Counts:")
print(df['action'].value_counts())

# Investigate the id column
print("\n--- Investigating User Actions ---")
# Find unique actions per user
user_actions = df.groupby('id')['action'].apply(set)

# How many viewers also clicked?
clickers_who_viewed = user_actions.apply(lambda x: 'view' in x and 'click' in x).sum()
print(f"Number of users who viewed and also clicked: {clickers_who_viewed}")

# Check for anomalies: users who clicked without viewing
users_who_only_clicked = user_actions.apply(lambda x: x == {'click'}).sum()
print(f"Number of users who clicked but never viewed (ANOMALY): {users_who_only_clicked}")

# Check for group overlap
control_ids = set(df[df['group'] == 'control']['id'].unique())
experiment_ids = set(df[df['group'] == 'experiment']['id'].unique())
overlap = control_ids & experiment_ids
print(f"Number of users present in both groups: {len(overlap)}")

# If there is overlap, how to account for it?
if len(overlap) > 0:
    print("\nWARNING: Group overlap detected! This invalidates the experiment.")
    print("Solution: The experiment should be stopped and restarted with proper randomization.")
else:
    print("\nNo group overlap detected. Experimental design is valid.")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None

Group Counts:
control       4264
experiment    3924
Name: group, dtype: int64

Action Counts:
view     6328
click    1860
Name: action, dtype: int64

--- Investigating User Actions ---
Number of users who viewed and also clicked: 1860
Number of users who clicked but never viewed (ANOMALY): 0
Number of users present in both groups: 0

No group overlap detected. Experimental design is valid.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
# Split the data into control and experiment groups
control_df = df[df['group'] == 'control']
experiment_df = df[df['group'] == 'experiment']
print(control_df)
print(experiment_df)

                       timestamp      id    group action
3     2016-09-24 19:59:02.646620  671993  control   view
9     2016-09-24 21:00:12.278374  560027  control   view
23    2016-09-25 00:25:14.141290  281985  control   view
24    2016-09-25 01:14:48.603202  407864  control   view
25    2016-09-25 02:16:11.046654  342984  control   view
...                          ...     ...      ...    ...
8178  2017-01-18 08:17:12.675797  616692  control   view
8181  2017-01-18 09:07:37.661143  795585  control   view
8182  2017-01-18 09:09:17.363917  795585  control  click
8186  2017-01-18 10:08:51.588469  505451  control   view
8187  2017-01-18 10:24:08.629327  461199  control   view

[4264 rows x 4 columns]
                       timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
4     2016-09-24 20:26:14.466886  536734  experimen

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
control_clicks = control_df['action'].value_counts().get('click', 0)
control_views = control_df['action'].value_counts().get('view', 0)
control_ctr = control_clicks / control_views

experiment_clicks = experiment_df['action'].value_counts().get('click', 0)
experiment_views = experiment_df['action'].value_counts().get('view', 0)
experiment_ctr = experiment_clicks / experiment_views

expected_experiment_clicks = control_ctr * experiment_views

print(f"Expected number of clicks for the experiment group: {expected_experiment_clicks}")


Expected number of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:
#Your code here
import math
# Calculate the standard deviation of the experiment group
std_dev_experiment = math.sqrt(experiment_ctr * (1 - experiment_ctr) / experiment_views)
print(f"Standard Deviation of Experiment Group: {std_dev_experiment}")

Standard Deviation of Experiment Group: 0.008447662249543294


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [6]:
#Your code here
# Calculate the z-score
z_score = (experiment_clicks - expected_experiment_clicks) / math.sqrt(control_ctr * (1 - control_ctr) / control_views + experiment_ctr * (1 - experiment_ctr) / experiment_views)
print(f"Z-Score: {z_score}")

Z-Score: 7837.105835955959


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
 based on the statistical analysis, it appears that the new homepage design did not provide a measurable improvement over the original. This suggests that the changes implemented may not have had the desired impact on user engagement.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.